In [1]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import re

In [2]:
os.chdir(u'D:\Terry\工作文件\氣象局計畫/資料\稻米災損資料')
os.getcwd()

'D:\\Terry\\工作文件\\氣象局計畫\\資料\\稻米災損資料'

In [3]:
df1 = pd.read_excel('98-107稻米資料.xlsx',sheet_name='各期種植面積')

In [4]:
df1

,年份,縣市,鄉鎮,一期插植時期,ㄧ期種植面積(公頃),二期插植時期,二期種植面積(公頃),種植面積合計
0,98.0,台北市,松山區,-,0.00,-,0.0,0.00
1,98.0,台北市,信義區,-,0.00,-,0.0,0.00
2,98.0,台北市,大安區,-,0.00,-,0.0,0.00
3,98.0,台北市,中山區,-,0.00,-,0.0,0.00
4,98.0,台北市,中正區,-,0.00,-,0.0,0.00
...,...,...,...,...,...,...,...,...
3821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df1.dropna(subset=['鄉鎮'],inplace=True)

In [16]:
df1['鄉鎮'] = df1['鄉鎮'].map(lambda x:re.sub('崁項鄉','崁頂鄉',x))
df1['鄉鎮'] = df1['鄉鎮'].map(lambda x:re.sub('礁溪鎮','礁溪鄉',x))

In [18]:
df1.rename(columns={'ㄧ期種植面積(公頃)':'一期種植面積(公頃)'},inplace=True)

In [20]:
df_1 = df1[['年份','縣市','鄉鎮','一期插植時期','一期種植面積(公頃)']]
df_1.rename(columns={'一期插植時期':'插植時期','一期種植面積(公頃)':'種植面積(公頃)'},inplace=True)
df_2 = df1[['年份','縣市','鄉鎮','二期插植時期','二期種植面積(公頃)']]
df_2.rename(columns={'二期插植時期':'插植時期','二期種植面積(公頃)':'種植面積(公頃)'},inplace=True)
df_1['作物名稱'] = '一期稻作'
df_2['作物名稱'] = '二期稻作'

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [25]:
output_df = pd.concat([df_1,df_2])

In [27]:
output_df['插植起始日_起'] = output_df.apply(lambda x:x['插植時期'].split('-')[0],axis=1)
output_df['插植起始日_末'] = output_df.apply(lambda x:x['插植時期'].split('-')[1],axis=1)
output_df['插植起始日_起'] = output_df.apply(lambda x:re.sub('月|日','',x['插植起始日_起']),axis=1)
output_df['插植起始日_末'] = output_df.apply(lambda x:re.sub('月|日','',x['插植起始日_末']),axis=1)

In [61]:
def date_convert(date,year):
    try:
        if len(date)==2:
            return datetime.datetime.strptime(str(int(year+1911))+'0'+date[0:1]+'0'+date[-1:], "%Y%m%d").date()
        elif len(date)==3:
            return datetime.datetime.strptime(str(int(year+1911))+'0'+date[0:1]+date[1:], "%Y%m%d").date()
        elif len(date)==4:
            return datetime.datetime.strptime(str(int(year+1911))+date, "%Y%m%d").date()    
        else:
            return ''
    except:
        if len(date)==2:
            return str(int(year+1911))+'0'+date[0:1]+'0'+date[-1:]
        elif len(date)==3:
            return str(int(year+1911))+'0'+date[0:1]+date[1:]
        elif len(date)==4:
            return str(int(year+1911))+date, "%Y%m%d"    
        else:
            return '' 
    
def check(start,end):
    try:
        if start<end:
            return 1
        else:
            return 0
    except:
        return 0
    
def date_correct(date,check,year):
    if check==0:   
        try:
            if len(date)==4:
                return datetime.datetime.strptime(str(int(year+1910))+date, "%Y%m%d").date()
            elif len(date)==3 and date[0:2]=='10':
                return datetime.datetime.strptime(str(int(year+1911))+date[0:2]+'0'+date[-1:], "%Y%m%d").date()
            else:
                return date_convert(date,year)
        except:
            return date
    else:
        return date_convert(date,year) 

In [49]:
output_df['插植起始日_起(date)'] = output_df.apply(lambda x:date_convert(x['插植起始日_起'],x['年份']),axis=1)
output_df['插植起始日_末(date)'] = output_df.apply(lambda x:date_convert(x['插植起始日_末'],x['年份']),axis=1)

In [53]:
output_df['check'] = output_df.apply(lambda x:check(x['插植起始日_起(date)'],x['插植起始日_末(date)']),axis=1)

In [54]:
output_df[(output_df['check']==0)&(output_df['插植起始日_起']!='')&(output_df['插植起始日_末']!='')]

,年份,縣市,鄉鎮,插植時期,種植面積(公頃),作物名稱,插植起始日_起,插植起始日_末,插植起始日_起(date),插植起始日_末(date),check
13,98.0,新北市,板橋區,311-311,0.09,一期稻作,311,311,2009-03-11,2009-03-11,0
19,98.0,新北市,樹林區,39-39,0.81,一期稻作,39,39,2009-03-09,2009-03-09,0
25,98.0,新北市,土城區,310-310,5.00,一期稻作,310,310,2009-03-10,2009-03-10,0
108,98.0,台中市,東區,220-220,0.14,一期稻作,220,220,2009-02-20,2009-02-20,0
170,98.0,南投縣,鹿谷鄉,212-230,41.58,一期稻作,212,230,2009-02-12,20090230,0
...,...,...,...,...,...,...,...,...,...,...,...
3071,106.0,南投縣,埔里鎮,81-730,17.14,二期稻作,81,730,2017-08-01,2017-07-30,0
3121,106.0,嘉義縣,竹崎鄉,830-830,0.97,二期稻作,830,830,2017-08-30,2017-08-30,0
3230,106.0,屏東縣,三地門鄉,620-620,0.44,二期稻作,620,620,2017-06-20,2017-06-20,0
3378,107.0,台中市,東區,815-815,0.06,二期稻作,815,815,2018-08-15,2018-08-15,0


In [63]:
output_df['插植起始日_起(date)'] = output_df.apply(lambda x:date_correct(x['插植起始日_起'],x['check'],x['年份']),axis=1)
output_df['插植起始日_末(date)'] = output_df.apply(lambda x:date_correct(x['插植起始日_末'],x['check'],x['年份']),axis=1)
output_df['check'] = output_df.apply(lambda x:check(x['插植起始日_起(date)'],x['插植起始日_末(date)']),axis=1)

In [64]:
output_df[(output_df['check']==0)&(output_df['插植起始日_起']!='')&(output_df['插植起始日_末']!='')]

,年份,縣市,鄉鎮,插植時期,種植面積(公頃),作物名稱,插植起始日_起,插植起始日_末,插植起始日_起(date),插植起始日_末(date),check
170,98.0,南投縣,鹿谷鄉,212-230,41.58,一期稻作,212,230,2009-02-12,20090230,0
202,98.0,嘉義縣,太保市,1223-39,2244.68,一期稻作,1223,39,2009-12-23,2009-03-09,0
203,98.0,嘉義縣,朴子市,1220-320,1505.42,一期稻作,1220,320,2009-12-20,2009-03-20,0
204,98.0,嘉義縣,布袋鎮,1222-320,758.60,一期稻作,1222,320,2009-12-22,2009-03-20,0
209,98.0,嘉義縣,六腳鄉,1227-320,469.53,一期稻作,1227,320,2009-12-27,2009-03-20,0
...,...,...,...,...,...,...,...,...,...,...,...
2755,105.0,嘉義縣,鹿草鄉,71-101,1761.20,二期稻作,71,101,2016-07-01,2016-01-01,0
2882,105.0,台東縣,長濱鄉,721-279,25.00,二期稻作,721,279,2016-07-21,20160279,0
2987,106.0,新竹縣,芎林鄉,84-716,332.39,二期稻作,84,716,2017-08-04,2017-07-16,0
3071,106.0,南投縣,埔里鎮,81-730,17.14,二期稻作,81,730,2017-08-01,2017-07-30,0


In [62]:
output_df.to_excel('稻米生育日期.xlsx',index=False)